In [ ]:
import pyVoxelFEM
import MeshFEM, mesh
import numpy as np
import matplotlib.pyplot as plt
from tri_mesh_viewer import QuadHexViewer

# TensorProductSimulator
This notebook shows how to define a linear elastic problem on a hexahedral domain and how to initialize a simulator to solve it.

In [ ]:
# 2D or 3D?
DIM = 3

# Material and "boundary conditions" (can be imposed also inside the domain) are read from file
MATERIAL_PATH = '../examples/materials/B9Creator.material'
if DIM == 2:
    BC_PATH = '../examples/bcs/cantilever_flexion_E.bc'
elif DIM == 3:
    BC_PATH = '../examples/bcs/3D/cantilever_flexion_E.bc'

# Domain and mesh definition
if DIM == 2:
    orderFEM = [1, 1]    # available: [1, 1] or [2, 2]
    domainCorners = [[0, 0], [2, 1]]
    gridDimensions = [4, 2]
elif DIM == 3:
    orderFEM = [1, 1, 1] # available: [1, 1, 1] or [2, 2, 2]
    domainCorners = [[0, 0, 0], [2, 1, 1]]
    gridDimensions = [4, 2, 2]

# Initialize simulator, define material and apply external constraints
tps = pyVoxelFEM.TensorProductSimulator(orderFEM, domainCorners, gridDimensions)
tps.readMaterial(MATERIAL_PATH)
tps.applyDisplacementsAndLoadsFromFile(BC_PATH)

# Update densities
uniformDensity = 1.0
tps.setUniformDensities(uniformDensity)
tps.setElementDensity(1, 0)

In [ ]:
# Get some TPS info
print("Densities: \n", tps.getDensities(), end='\n\n')
elemIndex = (np.array(gridDimensions)/2).astype(int)
print("Index of element in position " + str(elemIndex) + ": \n", tps.elementIndexForGridCell(elemIndex), end='\n\n')
print("Number of elements: \n", tps.numElements())

In [ ]:
# Show domain with the defined density distribution
view = QuadHexViewer(*tps.getMesh(), scalarField=tps.getDensities())
view.show()

In [ ]:
# Show imposed displacements
view.update(scalarField=tps.getDisplacementsIndicators())

In [ ]:
# Show external forces
view.update(scalarField=tps.getForcesIndicators())

In [ ]:
# Solve the elastic problem
u = tps.solveWithImposedLoads()

In [ ]:
# Export mesh and solution to `.msh` file
mfw = mesh.MSHFieldWriter('test.msh', *tps.getMesh())
mfw.addField('Density', tps.getDensities())
mfw.addField('u', u)
del mfw